패턴인식과머신러닝 Team Project Team C

1970014 김세빈

1970062 이서린

2070020 김정연

2070087 최지인

## 1. Import Library

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import torch
from torch import nn

import albumentations as albu
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, models

from torch.autograd import Variable
import torch.utils.data as data
import pandas as pd
import os
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset

import sklearn.model_selection as sk

import warnings
warnings.filterwarnings('ignore')

## 2. Import Data

In [2]:

# ######### 코랩 환경 ##########

from google.colab import drive
drive.mount('/content/drive')

# Dataset path load
train_path = '/content/drive/MyDrive/패턴/TrainData/Train'

Mounted at /content/drive


For least randomness, fix seed as 7

In [3]:
import random
seed = 7
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

## 3. Data Preprocessing

##3.1 Image Normalization, Data split

In [4]:
# Data Load
classes = ['campus', 'farm', 'train_station']  #campus:0, farm:1, train_station:2

data=[]
label=[]

## i : Integer representation for each classes (campus/farm/train_station -> 0/1/2)
for i,d in enumerate(classes):
    files = os.listdir(train_path+'/'+d)
    
    for f in files:
        img = Image.open(train_path+'/'+d+'/'+f)
        
        # Data normalization
        one_img = np.asarray(np.float32(img))
        norm_img = one_img/255.0

        img = np.asarray([norm_img])
        data.append(img)
        label.append(i)
        
data = np.array(data, dtype='float32')
label = np.array(label, dtype='int64')

total_X = torch.from_numpy(data)
total_Y = torch.from_numpy(label)

class별 3000장 균등 분배 확인, train data를 2배 증식할 것이기 때문에

 각 class 별 train 2000장 : test 1000장으로 data split을 해 train을 4000장으로 늘릴 예정. 
 
 따라서 data split에 test data 비율 0.333 적용.

 각 class별 train 2000장, test 1000장이므로 전체는 6000:3000장

In [5]:
#checking class
df1=pd.DataFrame(total_Y.numpy())

zero=0
one=0
two=0
for i in range(len(df1)):
  if int(df1.iloc[i])==0:
    zero+=1
  if int(df1.iloc[i])==1:
    one+=1
  if int(df1.iloc[i])==2:
    two+=1
print('Campus class 데이터 수 :', zero)
print('Farm class 데이터 수 :', one)
print('Train station 데이터 수 :', two)

# Dataset split
X_train, X_valid, Y_train, Y_valid = sk.train_test_split(total_X, total_Y, test_size=0.33333, random_state=100, stratify=total_Y)
print(X_train.shape, X_valid.shape, Y_train.shape, Y_valid.shape)

Campus class 데이터 수 : 3000
Farm class 데이터 수 : 3008
Train station 데이터 수 : 3000
torch.Size([6005, 1, 100, 100]) torch.Size([3003, 1, 100, 100]) torch.Size([6005]) torch.Size([3003])


##3.2 Augmentation

###3.2.1 Data Augmentation

Train dataset Horizontal flip

In [6]:
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

flip = transforms.RandomHorizontalFlip(p=1)
X_train_aug=flip(X_train)
X_train_new=torch.concat((X_train, X_train_aug), dim=0)
Y_train_new=torch.concat((Y_train,Y_train),dim=0)

print(X_train_new.shape, X_valid.shape, Y_train_new.shape, Y_valid.shape)

torch.Size([12010, 1, 100, 100]) torch.Size([3003, 1, 100, 100]) torch.Size([12010]) torch.Size([3003])


RandomChoice

###3.2.2 Image Augmentation

In [7]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode = mode
        
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            img = train_transform(image=img)
        
        if self.mode=='test':
            img = test_transform(image=img)
        
        label = self.labels[idx]
        return img, label

In [8]:
train_transform = albu.Compose([
    albu.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    albu.OneOf([
        albu.MotionBlur(p=0.2),
        albu.MedianBlur(blur_limit=3, p=0.1),
        albu.Blur(blur_limit=3, p=0.1),
    ], p=0.2),
])

# No transformation added to test(validation) dataset
test_transform = albu.Compose([
])

In [9]:
# DataLoader
train_dataset = Custom_dataset(np.array(X_train_new),np.array(Y_train_new), mode='train')
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

valid_dataset = Custom_dataset(np.array(X_valid),np.array(Y_valid), mode='test')
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

In [10]:
#Checking whether augmentation is done well and image dataset are  i

newdf=pd.DataFrame(Y_train_new.numpy())

zero=0
one=0
two=0
for i in range(len(newdf)):
  if int(newdf.iloc[i])==0:
    zero+=1
  if int(newdf.iloc[i])==1:
    one+=1
  if int(newdf.iloc[i])==2:
    two+=1
print('Campus class 데이터 수 :', zero)
print('Farm class 데이터 수 :', one)
print('Train station 데이터 수 :', two)

Campus class 데이터 수 : 4000
Farm class 데이터 수 : 4010
Train station 데이터 수 : 4000


## 4. Define Model

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(
            
            nn.Conv2d(1, 16, 3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            
            nn.Conv2d(16, 16, 3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            
            nn.MaxPool2d(3,2),
            
            nn.Conv2d(16, 32, 3),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.Conv2d(32, 32, 3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            nn.MaxPool2d(3,2),
            
            nn.Conv2d(32, 64, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            nn.Conv2d(64, 64, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            nn.MaxPool2d(3,2),

        )
        
  
        self.fc = nn.Sequential(
                nn.Linear(4096, 256),
                nn.BatchNorm1d(256),
                nn.ReLU(),
                nn.Dropout(p=0.3),
                nn.Linear(256, 128),
                nn.BatchNorm1d(128),
                nn.ReLU(),
                nn.Dropout(p=0.3),
                nn.Linear(128, 3)
                

        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x


model = Net()
model.cuda()

Net(
  (conv): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (11): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (15): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [12]:
def run_train(model):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.NAdam(model.parameters(), lr=0.001)
    
    total_loss = 0
    correct = 0
    incorrect = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs['image'].type(torch.cuda.FloatTensor), labels.type(torch.cuda.LongTensor)
        inputs, labels = Variable(inputs), Variable(labels)
        optimizer.zero_grad()
        output = model(inputs)
        
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.data.item()

        idx, pred = output.max(1)
        pred = (pred.detach().cpu().numpy())
        truth = (labels.detach().cpu().numpy())
        
        for ix in range(len(pred)):
            if pred[ix] == truth[ix]:
                correct = correct +1
            else:
                incorrect = incorrect +1
        accuracy = (correct/(correct+incorrect))*100
        train_img = inputs.cpu().numpy()
        
    return train_img, pred, truth, accuracy, total_loss

In [13]:
def run_valid(model):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    
    total_loss = 0
    correct = 0
    incorrect = 0
    
    for inputs, labels in valid_loader:
        inputs, labels = inputs['image'].type(torch.cuda.FloatTensor), labels.type(torch.cuda.LongTensor)
        inputs, labels = Variable(inputs), Variable(labels)

        output = model(inputs)
        idx, pred = output.max(1)
        pred = (pred.detach().cpu().numpy())
        truth = (labels.detach().cpu().numpy())

        
        loss = criterion(output, labels)
        total_loss += loss.data.item()
        
        for ix in range(len(pred)):
            if pred[ix] == truth[ix]:
                correct = correct +1
            else:
                incorrect = incorrect +1
        accuracy = (correct/(correct+incorrect))*100
        valid_img = inputs.cpu().numpy()
        
    return valid_img, pred, truth, accuracy, total_loss

## 5. Train Model

In [ ]:
'''
acc_t = []
acc_v = []


epoch = 100
print('***START EPOCH***')

for E in range(1, epoch+1):
    print(f'# Epoch : {E} /{epoch}')
    train_img, pred_t, truth_t, accuracy_t, loss_t = run_train(model)
    valid_img, pred_v, truth_v, accuracy_v, loss_v = run_valid(model)
    
    print('Train Accuracy : {}'.format(accuracy_t))
    print('Validation Accuracy : {}'.format(accuracy_v))
    acc_t.append(accuracy_t)
    acc_v.append(accuracy_v)
'''

loss_t = []
loss_v = []
acc_t = []
acc_v = []

epoch = 100
print('***START EPOCH***')

for E in range(1, epoch+1):
    print(f'# Epoch : {E} /{epoch}')
    train_img, pred_t, truth_t, accuracy_t, loss_train = run_train(model)
    valid_img, pred_v, truth_v, accuracy_v, loss_valid = run_valid(model)
    
    print('Train Accuracy : {}'.format(accuracy_t))
    print('Validation Accuracy : {}'.format(accuracy_v))
    
    loss_t.append(loss_train)
    loss_v.append(loss_valid)
    acc_t.append(accuracy_t)
    acc_v.append(accuracy_v)

In [ ]:
# 시각화
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))

# 정확도 그래프
plt.subplot(1, 2, 1)
plt.plot(range(len(acc_t)), acc_t, 'b', range(len(acc_v)), acc_v, 'r')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Train and Validation Accuracy')
blue_patch = mpatches.Patch(color='blue', label='Train Accuracy')
red_patch = mpatches.Patch(color='red', label='Validation Accuracy')
plt.legend(handles=[red_patch, blue_patch])

# 손실 함수 그래프
plt.subplot(1, 2, 2)
plt.plot(range(len(loss_t)), loss_t, 'b', range(len(loss_v)), loss_v, 'r')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train and Validation Loss')
blue_patch = mpatches.Patch(color='blue', label='Train Loss')
red_patch = mpatches.Patch(color='red', label='Validation Loss')
plt.legend(handles=[red_patch, blue_patch])

plt.tight_layout()
plt.show()

## 6. Save Model

In [46]:
########## 코랩 환경 ##########

# Save the weight matrices and bias vectors that will be loaded for testing later
torch.save(model,'/content/drive/MyDrive/패턴/TeamC')